In [12]:
import sys
print(sys.version)
import os

! eups list -s | grep pfs_pipe2d

ipynb_path = os.path.dirname(os.path.realpath("__file__"))
ipynb_path

3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
pfs_pipe2d            w.2022.17  	current setup


'/home/neven'

In [13]:
# make notebook nice and wide to fill the entire screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import numpy as np
import scipy.optimize
import time
import pandas as pd
import warnings

import psycopg2

from pfs.utils.fiberids import FiberIds  

# %matplotlib widget
%matplotlib inline
import matplotlib

import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.legend_handler import HandlerPatch
np.set_printoptions(suppress=True)

plt.rcParams["figure.facecolor"] = "white"
plt.rcParams['figure.figsize'] = (14, 10)
matplotlib.rcParams.update({'font.size': 10})

%load_ext autoreload
%autoreload 2

work_directory = '/work/ncaplar/BlackDotsData/'

conn = psycopg2.connect("dbname='opdb' host='pfsa-db01' port=5432 user='pfs' password=2394f4s3d") 

def getCobraGeometry(cobra_geometry_calib_id=None):
    if not cobra_geometry_calib_id:
        with conn:
             tmp = pd.read_sql(f'''
                SELECT DISTINCT 
                    max(cobra_geometry_calib_id) as cobra_geometry_calib_id
                FROM cobra_geometry_calib
                ''', conn)

        cobra_geometry_calib_id = tmp.cobra_geometry_calib_id[0]

    with conn:
        cobra_geometry = pd.read_sql(f'''
            SELECT DISTINCT 
                *
            FROM cobra_geometry
            WHERE cobra_geometry_calib_id = {cobra_geometry_calib_id}
            ''', conn)
        
    return cobra_geometry

getCobraGeometry_cobra_id=getCobraGeometry().set_index('cobra_id')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
with conn:
     df = pd.read_sql("select pfs_visit.pfs_visit_id, pfs_design_id, iic_sequence.visit_set_id, sequence_type from iic_sequence inner join "
                      "visit_set on visit_set.visit_set_id=iic_sequence.visit_set_id inner join pfs_visit on pfs_visit.pfs_visit_id=visit_set.pfs_visit_id "
                      "where sequence_type='thetaCrossing' or sequence_type='phiCrossing' ", conn)
df

,pfs_visit_id,pfs_design_id,visit_set_id,sequence_type
0,76680,8091575085665692756,14992,thetaCrossing
1,76681,8091575085665692756,14993,thetaCrossing
2,76682,8091575085665692756,14994,thetaCrossing
3,76685,2827351173347336962,14997,phiCrossing
4,76693,3550208719454350404,15001,thetaCrossing
5,76703,2166000543195576900,15003,thetaCrossing
6,76705,1438296865955205244,15005,phiCrossing
7,77482,2554769512037401971,15229,phiCrossing
8,77816,5426181922868053827,15517,phiCrossing
9,77873,5426181922868053827,15520,phiCrossing


In [15]:
#  run number 9  (June 20 2022, PFI)
mcs_data_all_17 = np.load('/work/ncaplar/HiloAnalysis/Cobra_data/mcsData.v077873.phi.npy')
mcs_data_all_18 = np.load('/work/ncaplar/HiloAnalysis/Cobra_data/mcsData.v077875.theta.npy')


In [17]:
type(mcs_data_all_17)

numpy.ndarray